# Walkthrough of Signal/Slot Code in Prismatic Qt
### Luis Rangel DaCosta, 6-27-2020
based off of code at lerandc/prismatic master branch @ commit  849cc84826a6a484c44bc6a33f9d99e8af2483e1

We're going to walk through each type of signal/slot currently used in the Prismatic GUI and how they are implemented, using an example of each. We'll start with one of the simplest features, the button.

# Button
## example: btn_reset
resets the state of the calculation

## Widget code in prismqtmainwindow.ui
This code is generated automatically by the Qt creator editor. The actual visual component and interactivity of the button is defined here

From: prismmainwindow.ui line 4585
```
<widget class="QPushButton" name="btn_reset">
 <property name="enabled">
  <bool>true</bool>
 </property>
 <property name="minimumSize">
  <size>
   <width>0</width>
   <height>50</height>
  </size>
 </property>
 <property name="maximumSize">
  <size>
   <width>16777215</width>
   <height>16777215</height>
  </size>
 </property>
 <property name="font">
  <font>
   <family>Roboto</family>
   <pointsize>14</pointsize>
  </font>
 </property>
 <property name="layoutDirection">
  <enum>Qt::RightToLeft</enum>
 </property>
 <property name="text">
  <string>Reset Calculation</string>
 </property>
</widget>
```

## Connection to Slot
This code is manually created. Loosely, this code can be understood like this: if the GUI window gets a notification `clicked()` from the button called `btn_reset`, run the code `resetCalculation()`.

From: prismmainwindow.cpp line 382
```
connect(this->ui->btn_reset, SIGNAL(clicked()), this, SLOT(resetCalculation()));
```

Let's break this down a bit more though. From the Qt5 documentation (https://wiki.qt.io/New_Signal_Slot_Syntax), the syntax for `connect` looks like this:
```
connect(
    sender, SIGNAL( valueChanged( QString, QString ) ),
    receiver, SLOT( updateValue( QString ) )
);
```

### Sender

`sender` :: `this->ui->btn_reset`

`this` refers to the parent object held by the host thread. When we started up the GUI, we exectuted the following code (from `prismatic/Qt/main.cpp')
```
PRISMMainWindow w;
    w.show();
```

Here, we are declaring a `PRISMMainWindow` object (whose class is defined in `prismmainwindow.h`). When the object is declared, it is set up by the class's constructor function, and when we are in the scope of the explicit constructor `explicit PRISMMainWindow(QWidget *parent = 0);` (which is where all the signals and slots are connected), `this` refers to the primary GUI object. 

`this` then looks at its private member `ui`, which is an object whose class is defined in `prismmainwindow.ui`. `ui` now looks for a member called `btn_reset` and finds the widget code we looked at above. 

So, in the end, the `sender` is the reset button that is a member of the UI object that is a member of the PRISMMainWindow object that was created when the GUI execution began.

### Signal

`SIGNAL( valueChanged( QString, QString ) )` :: `SIGNAL(clicked())`

We've now established that we're listening to the `sender`, `btn_reset`, for *something*. Here, we define that something as `clicked()`. This, of course, is clear to us what this means-- whenever the button is clicked, that's the signal. For comprehensiveness, though, let's take a look at where this signal comes from.

We can see that in the widget code, the button was created as an object usign the class `QPushButton`. From the API reference for Qt (https://doc.qt.io/qt-5/qpushbutton.html), we can see what member functions this class has. All of the members are listed here: https://doc.qt.io/qt-5/qpushbutton-members.html. `clicked()` looks like an inherited member from `QAbstractButton`, which defines the state as:

>void QAbstractButton::clicked(bool checked = false)
>>This signal is emitted when the button is activated (i.e., pressed down then released while the mouse cursor is inside the button), when the shortcut key is typed, or when click() or animateClick() is called. Notably, this signal is not emitted if you call setDown(), setChecked() or toggle().
>>
>>If the button is checkable, checked is true if the button is checked, or false if the button is unchecked.

Looking at the `QAbstractButton` class, we can also see that there are three other signals that we could use in the `SIGNAL()` call-- `pressed()`, `released()`, and `toggled()`.

### Receiver

`receiver` :: `this`

Similar to the `sender`, `this`, in the context of the constructor call, is referring to the object that is being constructed, the `PRISMMainWindow`. Here, it's just the object is listening to itself (in fact, this is all the signals and slots for our GUI- there is just a single, monolithic GUI object existing in its own little ecosystem). You could imagine, however, in a more complex GUI program that there are seperate objects created for different features that exist in their own space and time and connect to eachother and communicate through these signals and slots through some sort of networking paradigm.

### Slot

`SLOT( updateValue( QString ) )` :: `SLOT(resetCalculation()))`

Here, the slot is just a function that should be called when the signal is heard by the `receiver`. All the slots available are defined in the `PRISMMainWindow` class as members that are all `public slots`. These are all `void` functions, and since they are members of the object, it's important to note that they can access the parent object through the same `this->` operator.

## Execution Code (the Slot)

All the member functions are defined in `prismmainwindow.cpp` using the class namespace `PRISMMainWindow`.

From prismmainwindow.cpp, line 2306:
```
void PRISMMainWindow::resetCalculation(){
    QMutexLocker gatekeeper(&dataLock);
    std::cout << "Resetting Calculation" << std::endl;
    outputReady     = false;
    ScompactReady   = false;
    potentialReady  = false;
    probeSetupReady = false;
}
```

We can see here what `resetCalculation()` now does. First, it locks the data that is owned by the parent object by using the `QMutexLocker` on the `dataLock` Qmutex (a private member of the parent). Then, it changes the state of `outputReady`, `ScompactReady`, `potentialReady`, and `probeSetupReady` to false (also all private members). Like mentioned above, each of these slots, as members of the parent `PRISMMainWindow` object, can access the members of the parent object.